In [ ]:
# Install GDAL and Geopandas
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!pip install git+git://github.com/geopandas/geopandas.git --quiet
!pip install descartes --quiet
!pip install geopy
!pip install plotly_express
!pip install ipython-autotime


Reading package lists...
Building dependency tree...
Reading state information...
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources
Suggested packages:
  python3-setuptools
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources python3-rtree
0 up

* Restart the run time. tqdm does not work with recent python so we downgrade it to get it working

In [ ]:
!pip install tqdm==4.36.0

     |████████████████████████████████| 61kB 3.7MB/s 
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.36.0 which is incompatible.
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [ ]:
%reload_ext autotime
import pandas as pd

import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

## Reverse Geocoding Single Example

In [ ]:
locator = Nominatim(user_agent="myGeocoder")
coordinates = "22.52014, 88.31144"
location = locator.reverse(coordinates)

time: 54.2 ms


In [ ]:
location.raw

{'address': {'city': 'Kolkata',
  'country': 'India',
  'country_code': 'in',
  'county': 'Kolkata',
  'postcode': '700060',
  'state': 'West Bengal',
  'state_district': 'Kolkata',
  'suburb': 'Mominpur'},
 'boundingbox': ['22.5200622', '22.5221258', '88.3049641', '88.3142989'],
 'display_name': 'Mominpur, Kolkata, West Bengal, 700060, India',
 'lat': '22.52066659051273',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'lon': '88.3115560342271',
 'osm_id': 206656217,
 'osm_type': 'way',
 'place_id': 133744786}

time: 4.56 ms


In [ ]:
print(location.address)

Mominpur, Kolkata, West Bengal, 700060, India
time: 1.94 ms


In [ ]:
location.raw["display_name"]

'Mominpur, Kolkata, West Bengal, 700060, India'

time: 3.29 ms


## Reverse Geocoding with Pandas

In [ ]:
from google.colab import files
uploaded = files.upload()


MessageError: ignored

time: 6.32 s


In [ ]:
import io
data = io.BytesIO(uploaded['geocoding1.csv']) 


In [ ]:
import pandas as pd 
cols = ["year_est","id", "firm","place", "district","state","lat","long"]  
df= pd.read_csv(data, usecols=cols, encoding = "ISO-8859-1")
df.head()

In [ ]:
print(df)

In [ ]:
df.shape

In [ ]:
import plotly.express as px
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")
fig = px.scatter_mapbox(df, lat="lat", lon="long",color="firm",
                   size_max=15, zoom=10)
fig.show()

In [ ]:
df["geom"] =  df["long"].map(str)  + ',' + df['lat'].map(str)
df.head()

In [ ]:
df.geom[0]



In [ ]:
locator = Nominatim(user_agent="myGeocoder", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)
rgeocode

In [ ]:
tqdm.pandas()
df['address'] = df['geom'].progress_apply(rgeocode)

In [ ]:
df.tail()


In [ ]:
df["address"][350]